In [1]:
import numpy as np #Otherwise ...

Funções para o caso tridiagonal

In [2]:
def ludcmp_tri(a, b, c):
  # Given a tridiagonal matrix stored in vectors a, b, and c, compute its LU
  # decomposition and stores in the same vectors. All vectors have the same
  # size. The lower diagonal is stored in a, with zero in the first component,
  # the main diagonal is stored in b, and the upper diagonal is stored in c,
  # with zero in the last element:
  #
  # a = [0, a_1, ..., a_{n-1}]
  #
  # b = [b_0, ..., b_{n-1}]
  #
  # c = [c_0, ..., c_{n-2}, 0]
  #
  # The multipliers l are returned on and the diagonal of U is returned on b.
  # This routine operates on "methods" for numpy arrays, so it behaves like
  # passage  as reference for the parameters.
  
  
  n = len(a)
  for i in range(1, n):
    a[i] /= b[i-1]  # multiplicador
    b[i] -= a[i]*c[i-1] # diagonal de U
    
def ludcmp_tri_solve(l, u, c, d):
  # Given u and l from the LU decomposition of a tridiagonal matrix A, and
  # the rhs d, solves Ax = d. The result is returned in d.
  
  
  n = len(l)
  
  # Ly = d; result in d
  for i in range(1, n):
    d[i] -= l[i]*d[i-1]
  
  # Ux = d; result in d
  d[n-1] /= u[n-1]
  for i in range(n-2, -1, -1):
    d[i] -= c[i]*d[i+1]; d[i] /= u[i]

Sistemas tridiagonais cíclicos

In [3]:
def cyclic_tridiagonal(a, b, c, d):
    #Given a cyclic tridiagonal matrix A characterized by the vectors a, b,
    #and c, solves Ax = d using the functions in module tridiagonal.py

    #Tridiagonal submatrix T
    l = np.copy(a[:-1]); l[0] = 0.0
    u = np.copy(b[:-1])
    cc = np.copy(c[:-1]); cc[-1] = 0.0

    ludcmp_tri(l, u, cc) #LU decomposition of the tridiagonal submatrix T

    #Solution of the first tridiagonal subsystem
    y = np.copy(d[:-1])
    ludcmp_tri_solve(l, u, cc, y)

    #Solution of the second tridiagonal subsystem
    z = np.zeros(len(a)-1); z[0] = a[0]; z[-1] = c[-2]
    ludcmp_tri_solve(l, u, cc, z)

    #Solution of the cyclic system
    x = np.empty(len(a)) #Allocate memory
    x[-1] = (d[-1]-c[-1]*y[0]-a[-1]*y[-1])/(b[-1]-c[-1]*z[0]-a[-1]*z[-1])
    x[:-1] = y - x[-1]*z

    print("L: ")
    print(l)
    print("\n U: ")
    print(u)

    return x

Exemplo da Tarefa 1

In [9]:
n = 5
iv = np.arange(n) + 1.0 #Vetor formado pelos índices 1,...,n
a = np.empty(n); a[:-1] = (2*iv[:-1] - 1) / (4*iv[:-1]); a[-1] = (2*iv[-1] - 1) / (2*iv[-1]) #Vetor a
c = 1 - a #Vetor c
b = 2*np.ones(n) #Vetor b
d = np.cos(2*np.pi*iv*iv/(n*n)) #Lado direito so sistema tridiagonal cíclico

print("\n%8s %10s %11s %11s\n" % ('a', 'b', 'c', 'd')) #Veja os dados
for i in range(n):
    print("%11.8f %11.8f %11.8f %11.8f" % (a[i], b[i], c[i], d[i]))
print("\n")


       a          b           c           d

 0.25000000  2.00000000  0.75000000  0.96858316
 0.37500000  2.00000000  0.62500000  0.53582679
 0.41666667  2.00000000  0.58333333 -0.63742399
 0.43750000  2.00000000  0.56250000 -0.63742399
 0.90000000  2.00000000  0.10000000  1.00000000




In [10]:

x = cyclic_tridiagonal(a, b, c, d) #Solução

print("\n%10s\n" %('x',)) #Impressão do resultado
for i in range(n):
    print("%19.16f" % x[i])
print("\n")

L: 
[0.         0.1875     0.22408964 0.23522214]

 U: 
[2.         1.859375   1.85994398 1.86278709]

         x

 0.2888525037465419
 0.2903382545973282
-0.2450710449929809
-0.4598677243056251
 0.6924978507502042




Verificação com o numpy, sem explorar a estrutura da matriz

In [6]:
A = np.diag(a[1:],k=-1) + np.diag(b) + np.diag(c[:-1],k=1); A[0,-1] = a[0]; A[-1,0] = c[-1] #Matriz cíclica
xx = np.linalg.solve(A, d) #Solução de Ax = d usando numpy
erro = abs((xx-x)/xx) #Erro relativo componente a componente
print("\nErro = %.1e\n" % max(erro))


Erro = 1.9e-16

